In [1]:
#import required libraries
#pandas to make and manipulate dataframes
#numpy for the technical analysis
import pandas as pd
import yfinance as yf
import numpy as np
import math
import panel as pn
pn.extension('plotly')
#import pandas_ta as ta
import hvplot.pandas
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
# parameter setup (default values in the original indicator)
#These are the constants to be used for the moving averages, the bollinger bands and the keltner channels
#ultimately these will be needed for the squeeze and other calculations
length = 20
mult = 2
length_KC = 21
mult_KC = 2

In [3]:
# With this code you can input any stock and the data for the stock will be pulled from the Yahoo Finance API using yf.Ticker
#The data is pulled for 3 years with a daily timeframe and put into a dataframe called df_stock
#This data is only for the market so when entering the stock symbol use DIA, QQQ, SPY, and major market indices
value = input("Enter Stock Symbol:")
period = input('enter period of data eg 1y or 2y:')
stock = yf.Ticker(value)
print(stock.info)
stock_df = stock.history(period=period, interval='1d')
stock_df.index.duplicated().sum()
df_stock = stock_df.loc[~stock_df.index.duplicated(keep='first')]
df_stock

Enter Stock Symbol: ma
enter period of data eg 1y or 2y: 2y


{'address1': '2000 Purchase Street', 'city': 'Purchase', 'state': 'NY', 'zip': '10577', 'country': 'United States', 'phone': '914 249 2000', 'website': 'https://www.mastercard.com', 'industry': 'Credit Services', 'industryKey': 'credit-services', 'industryDisp': 'Credit Services', 'sector': 'Financial Services', 'sectorKey': 'financial-services', 'sectorDisp': 'Financial Services', 'longBusinessSummary': 'Mastercard Incorporated, a technology company, provides transaction processing and other payment-related products and services in the United States and internationally. It facilitates the processing of payment transactions, including authorization, clearing, and settlement, as well as delivers other payment-related products and services. The company offers integrated products and value-added services for account holders, merchants, financial institutions, businesses, governments, and other organizations, such as programs that enable issuers to provide consumers with credits to defer p

Open        High         Low       Close  \
Date                                                                        
2022-01-27 00:00:00-05:00  344.851123  355.273335  330.279809  346.283569   
2022-01-28 00:00:00-05:00  354.700311  378.132959  351.074788  377.876129   
2022-01-31 00:00:00-05:00  374.092548  381.867203  371.731517  381.699280   
2022-02-01 00:00:00-05:00  381.086758  390.708754  377.787223  387.310425   
2022-02-02 00:00:00-05:00  384.574045  395.075276  378.113264  391.943665   
...                               ...         ...         ...         ...   
2024-01-22 00:00:00-05:00  438.980011  440.940002  437.260010  439.750000   
2024-01-23 00:00:00-05:00  440.269989  440.600006  437.859985  438.760010   
2024-01-24 00:00:00-05:00  438.529999  440.929993  435.760010  438.700012   
2024-01-25 00:00:00-05:00  436.000000  438.630005  434.459991  436.799988   
2024-01-26 00:00:00-05:00  432.549988  439.890015  430.670013  438.529999   

                            Volume  Dividends  Stock Splits  
Date                                                         
2022-01-27 00:00:00-05:00  8416100        0.0           0.0  
2022-01-28 00:00:00-05:00  8813400        0.0           0.0  
2022-01-31 00:00:00-05:00  6345200        0.0           0.0  
2022-02-01 00:00:00-05:00  5841100        0.0           0.0  
2022-02-02 00:00:00-05:00  6115000        0.0           0.0  
...                            ...        ...           ...  
2024-01-22 00:00:00-05:00  2715500        0.0           0.0  
2024-01-23 00:00:00-05:00  2328600        0.0           0.0  
2024-01-24 00:00:00-05:00  2725700        0.0           0.0  
2024-01-25 00:00:00-05:00  2977800        0.0           0.0  
2024-01-26 00:00:00-05:00  2725400        0.0           0.0  

[502 rows x 7 columns]

In [4]:
import yahoo_fin.stock_info as si

In [5]:
help(si.get_earnings_history)

Help on function get_earnings_history in module yahoo_fin.stock_info:

get_earnings_history(ticker)
    Inputs: @ticker
    Returns the earnings calendar history of the input ticker with 
    EPS actual vs. expected data.



In [6]:
stock_earnings_hist = si.get_earnings_history(value)

TypeError: string indices must be integers, not 'str'

In [8]:
stock_earnings_hist

NameError: name 'stock_earnings_hist' is not defined

In [9]:
import pandas as pd
 
frame = pd.DataFrame.from_dict(stock_earnings_hist)
frame.to_csv(f'{value}_earnings_dates.csv')

NameError: name 'stock_earnings_hist' is not defined

In [14]:
# Python code to illustrate Sending mail with attachments 
# from your Gmail account  
  
# libraries to be imported 
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 
   
fromaddr = "akkrishn1@gmail.com"
toaddr = "fastandtheferocious@gmail.com"
   
# instance of MIMEMultipart 
msg = MIMEMultipart() 
  
# storing the senders email address   
msg['From'] = fromaddr 
  
# storing the receivers email address  
msg['To'] = toaddr 
  
# storing the subject  
msg['Subject'] = f"{value} earnings dates"
  
# string to store the body of the mail 
body = f"Here is the earnings dates for {value}"
  
# attach the body with the msg instance 
msg.attach(MIMEText(body, 'plain')) 
  
# open the file to be sent  
filename = f'{value}_earnings_dates.csv'
attachment = open(f"C:/Users/akkrishn/SqueezeMeBabyOneMoreTime/{value}_earnings_dates.csv", "rb") 
  
# instance of MIMEBase and named as p 
p = MIMEBase('application', 'octet-stream') 
  
# To change the payload into encoded form 
p.set_payload((attachment).read()) 
  
# encode into base64 
encoders.encode_base64(p) 
   
p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
  
# attach the instance 'p' to instance 'msg' 
msg.attach(p) 
  
# creates SMTP session 
s = smtplib.SMTP('smtp.gmail.com', 587) 
  
# start TLS for security 
s.starttls() 
  
# Authentication 
s.login(fromaddr, "Kr15hn40709") 
  
# Converts the Multipart msg into a string 
text = msg.as_string() 
  
# sending the mail 
s.sendmail(fromaddr, toaddr, text) 
  
# terminating the session 
s.quit()

(221, b'2.0.0 closing connection h2sm2077386pfv.35 - gsmtp')

In [10]:
si.get_stats_valuation(value, headers={'User-agent': 'Mozilla/5.0'})

Unnamed: 0 As of Date: 12/19/2023Current 9/30/2023  \
0      Market Cap (intraday)                       398.02B   373.03B   
1           Enterprise Value                       406.09B   382.14B   
2               Trailing P/E                         37.00     37.10   
3                Forward P/E                         29.85     27.25   
4  PEG Ratio (5 yr expected)                          1.43      1.35   
5          Price/Sales (ttm)                         16.59     16.09   
6           Price/Book (mrq)                         63.09     67.54   
7   Enterprise Value/Revenue                         16.67     58.49   
8    Enterprise Value/EBITDA                         27.55     92.68   

  6/30/2023 3/31/2023 12/31/2022 9/30/2022  
0   370.88B   344.51B    334.33B   273.82B  
1   379.48B   351.13B    340.82B   281.94B  
2     39.29     35.56      34.74     28.81  
3     32.05     29.67      28.41     22.47  
4      1.58      1.47       1.31      1.04  
5     16.63     15.87      15.70     13.39  
6     69.58     54.70      52.26     44.37  
7     60.53     61.09      58.59     48.98  
8     93.93    110.56     100.24     83.49

In [11]:
si.get_stats(value)

AttributeError: 'DataFrame' object has no attribute 'append'

In [8]:
stats = si.get_stats(value, headers={'User-agent': 'Mozilla/5.0'})
stats.to_csv(f'{value}_stats.csv')

In [15]:
# Python code to illustrate Sending mail with attachments 
# from your Gmail account  
  
# libraries to be imported 
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 
   
fromaddr = "akkrishn1@gmail.com"
toaddr = "fastandtheferocious@gmail.com"
   
# instance of MIMEMultipart 
msg = MIMEMultipart() 
  
# storing the senders email address   
msg['From'] = fromaddr 
  
# storing the receivers email address  
msg['To'] = toaddr 
  
# storing the subject  
msg['Subject'] = f"{value} stats info"
  
# string to store the body of the mail 
body = f"Here is the stats info {value}"
  
# attach the body with the msg instance 
msg.attach(MIMEText(body, 'plain')) 
  
# open the file to be sent  
filename = f'{value}_stats.csv'
attachment = open(f"C:/Users/akkrishn/SqueezeMeBabyOneMoreTime/{value}_stats.csv", "rb") 
  
# instance of MIMEBase and named as p 
p = MIMEBase('application', 'octet-stream') 
  
# To change the payload into encoded form 
p.set_payload((attachment).read()) 
  
# encode into base64 
encoders.encode_base64(p) 
   
p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
  
# attach the instance 'p' to instance 'msg' 
msg.attach(p) 
  
# creates SMTP session 
s = smtplib.SMTP('smtp.gmail.com', 587) 
  
# start TLS for security 
s.starttls() 
  
# Authentication 
s.login(fromaddr, "Kr15hn40709") 
  
# Converts the Multipart msg into a string 
text = msg.as_string() 
  
# sending the mail 
s.sendmail(fromaddr, toaddr, text) 
  
# terminating the session 
s.quit()

(221, b'2.0.0 closing connection p10sm31164422pfw.87 - gsmtp')

In [9]:
from yahoo_fin import options
import plotly.graph_objects as go
import pandas as pd

def list_of_expiry_dates(value):
    exp_dates = options.get_expiration_dates(value)
    #days = exp_dates - today_date
    return exp_dates

list_of_expiry_dates(value)

['March 16, 2022',
 'March 18, 2022',
 'March 21, 2022',
 'March 23, 2022',
 'March 25, 2022',
 'March 28, 2022',
 'March 30, 2022',
 'March 31, 2022',
 'April 1, 2022',
 'April 4, 2022',
 'April 6, 2022',
 'April 8, 2022',
 'April 11, 2022',
 'April 13, 2022',
 'April 14, 2022',
 'April 18, 2022',
 'April 22, 2022',
 'April 29, 2022',
 'May 20, 2022',
 'June 17, 2022',
 'June 30, 2022',
 'July 15, 2022',
 'August 19, 2022',
 'September 16, 2022',
 'September 30, 2022',
 'December 16, 2022',
 'December 30, 2022',
 'January 20, 2023',
 'March 17, 2023',
 'June 16, 2023',
 'December 15, 2023',
 'January 19, 2024',
 'December 20, 2024']

In [10]:
expiry = input('enter expiry date eg 12/17/2021')
def get_option_chain_calls(value):
    option_chain = options.get_options_chain(value)
    calls_chain = options.get_calls(value, expiry)
    return calls_chain

enter expiry date eg 12/17/2021 03/16/2022


In [11]:
def get_option_chain_puts(value):
    option_chain = options.get_options_chain(value)
    puts_chain = options.get_puts(value, expiry)
    return puts_chain

In [12]:
df_calls = get_option_chain_calls(value)
df_calls.to_csv(f'{value}_calls.csv')

In [13]:
df_puts = get_option_chain_puts(value)
df_puts.to_csv(f'{value}_puts.csv')

In [ ]:
# Python code to illustrate Sending mail with attachments 
# from your Gmail account  
  
# libraries to be imported 
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 
   
fromaddr = "akkrishn1@gmail.com"
toaddr = "fastandtheferocious@gmail.com"
   
# instance of MIMEMultipart 
msg = MIMEMultipart() 
  
# storing the senders email address   
msg['From'] = fromaddr 
  
# storing the receivers email address  
msg['To'] = toaddr 
  
# storing the subject  
msg['Subject'] = f"Option Chains {value}"
  
# string to store the body of the mail 
body = f"Here is the call option chain for {value}"
  
# attach the body with the msg instance 
msg.attach(MIMEText(body, 'plain')) 
  
# open the file to be sent  
filename = f'{value}_calls.csv'
attachment = open(f"C:/Users/akkrishn/SqueezeMeBabyOneMoreTime/SPY Watchlist/{value}_calls.csv", "rb") 
  
# instance of MIMEBase and named as p 
p = MIMEBase('application', 'octet-stream') 
  
# To change the payload into encoded form 
p.set_payload((attachment).read()) 
  
# encode into base64 
encoders.encode_base64(p) 
   
p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
  
# attach the instance 'p' to instance 'msg' 
msg.attach(p) 
  
# creates SMTP session 
s = smtplib.SMTP('smtp.gmail.com', 587) 
  
# start TLS for security 
s.starttls() 
  
# Authentication 
s.login(fromaddr, "Kr15hn40709") 
  
# Converts the Multipart msg into a string 
text = msg.as_string() 
  
# sending the mail 
s.sendmail(fromaddr, toaddr, text) 
  
# terminating the session 
s.quit() 

In [ ]:
# Python code to illustrate Sending mail with attachments 
# from your Gmail account  
  
# libraries to be imported 
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 
   
fromaddr = "akkrishn1@gmail.com"
toaddr = "fastandtheferocious@gmail.com"
   
# instance of MIMEMultipart 
msg = MIMEMultipart() 
  
# storing the senders email address   
msg['From'] = fromaddr 
  
# storing the receivers email address  
msg['To'] = toaddr 
  
# storing the subject  
msg['Subject'] = f"Option Chains {value}"
  
# string to store the body of the mail 
body = f"Here is the puts option chain for {value}"
  
# attach the body with the msg instance 
msg.attach(MIMEText(body, 'plain')) 
  
# open the file to be sent  
filename = f'{value}_puts.csv'
attachment = open(f"C:/Users/akkrishn/SqueezeMeBabyOneMoreTime/SPY Watchlist/{value}_puts.csv", "rb") 
  
# instance of MIMEBase and named as p 
p = MIMEBase('application', 'octet-stream') 
  
# To change the payload into encoded form 
p.set_payload((attachment).read()) 
  
# encode into base64 
encoders.encode_base64(p) 
   
p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
  
# attach the instance 'p' to instance 'msg' 
msg.attach(p) 
  
# creates SMTP session 
s = smtplib.SMTP('smtp.gmail.com', 587) 
  
# start TLS for security 
s.starttls() 
  
# Authentication 
s.login(fromaddr, "Kr15hn40709") 
  
# Converts the Multipart msg into a string 
text = msg.as_string() 
  
# sending the mail 
s.sendmail(fromaddr, toaddr, text) 
  
# terminating the session 
s.quit() 